In [1]:
import pandas as pd
import sys
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.score import MARSCore
from custom_score.utils import cleanString
from datasets_loaders.loaders import load_billsum
from sklearn.cluster import SpectralClustering, AffinityPropagation
from transformers import BertModel, BertTokenizer

import matplotlib.pyplot as plt
import numpy as np

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotat


### Billsum experimentation

In [ ]:
dataset = load_billsum()
subset = dataset.iloc[:2, :]

In [ ]:
#ms = MARSCore(subset["text"].to_list(), subset["summary"].to_list(), clusterizer=SpectralClustering(affinity='nearest_neighbors'))
ms = MARSCore(subset["text"].to_list(), subset["summary"].to_list())
ms.compute()

In [ ]:
_=ms.assess()

In [ ]:
print(ms)

### Pubmed experimentation

In [2]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

In [3]:
subset = dataset.iloc[3:5, :]
subset

,text,summary
3,congenital adrenal hyperplasia ( cah ) refers ...,congenital adrenal hyperplasia is a group of a...
4,type 1 diabetes ( t1d ) results from the destr...,objective(s):pentoxifylline is an immunomodula...


In [4]:
ms = MARSCore(subset["text"].to_list(),
              subset["summary"].to_list(),
              precision_level="s",
              ratio=5,
              n_allowed_elements=7,
              extraction_method="concat_l4",
              model=BertModel.from_pretrained('aitslab/biobert_huner_gene_v1'),
              tokenizer=BertTokenizer.from_pretrained('aitslab/biobert_huner_gene_v1'),
              clusterizer=AffinityPropagation(random_state=0))
ms.compute()

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\sklearn\cluster\_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


{0: {'tokens': ['congenital', '##ren', '##al', 'hyper', '##pl', '##asia', '##h', 'refers', 'group', 'auto', '##som', '##al', 'recess', '##ive', 'disorders', 'caused', 'enzyme', 'deficiency', 'which', 'leads', 'defects', 'bio', '##sy', '##nt', '##hesis', 'ste', '##roid', 'precursor', '##s', 'hydro', '##xy', '##lase', 'deficiency', 'precursor', '##s'], 'embs': array([[9.120947 , 5.135469 ],
       [9.2007675, 5.247996 ],
       [9.08703  , 5.119621 ],
       [9.100279 , 5.091368 ],
       [9.034446 , 5.0949597],
       [9.043628 , 5.147737 ],
       [9.030119 , 5.0488086],
       [8.894938 , 5.082359 ],
       [8.922773 , 5.043385 ],
       [8.802418 , 5.124179 ],
       [8.84947  , 5.1100535],
       [8.824309 , 5.0987415],
       [8.85481  , 5.105905 ],
       [8.768857 , 5.1031384],
       [8.710176 , 4.986864 ],
       [8.6458645, 5.0266166],
       [8.52942  , 5.075102 ],
       [8.5250435, 5.0628333],
       [8.595434 , 5.0569725],
       [8.433007 , 5.055581 ],
       [8.255237 , 

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\sklearn\cluster\_affinity_propagation.py:143: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


[(2, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (3, 2), (11, 2), (1, 2), (2, 2), (56, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (7, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (14, 2), (21, 2), (5, 2), (28, 2), (1, 2), (27, 2), (2, 2), (3, 2), (2, 2), (1, 2), (29, 2), (3, 2), (37, 2), (114, 2), (45, 2), (72, 2), (62, 2), (5, 2), (2, 2), (66, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (4, 2), (14, 2), (1, 2), (1, 2), (1, 2), (38, 2), (2, 2), (1, 2), (1, 2), (1, 2), (5, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (1, 2), (17, 2), (3, 2), (1, 2), (1, 2), (3, 2), (4, 2), (1, 2), (1, 2), (1, 2), (1, 2), (25, 2), (1, 2), (1, 2), (1, 2), (2, 2), (4, 2), (34, 2), (27, 2), (1, 2), (1,

QhullError: QH6214 qhull input error: not enough points(2) to construct initial simplex (need 3)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 700125188  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _maxoutside  0


In [ ]:
ms.summaries

In [ ]:
_=ms.assess()

In [ ]:
plt.imshow(ms.similarity_matrices[0], cmap='hot', interpolation='nearest')
plt.show()

In [ ]:
print(ms)

In [ ]:
ms.visualize(dim=2)

In [ ]:
print(ms.cluster_distribution())

### Spectral Clustering - Mincut

In [ ]:
from sklearn.cluster import SpectralClustering
from MARScore.utils import *
import hdbscan

In [ ]:
v = ms.vectors
l = ms.labels

In [ ]:
clusterer = SpectralClustering(affinity='nearest_neighbors')
v_clustered = clusterer.fit(v[0])


In [ ]:
v_clustered.labels_.astype(int)

In [ ]:
clusterer.__module__

In [ ]:
hdbscan.HDBSCAN().__module__

### Saving similarity Matrix

In [ ]:
import csv

def saveSimilarityMatrix(path, mat):
    with open(path, 'w', newline='') as f:
        write = csv.writer(f)
        write.writerows(mat)

In [ ]:
saveSimilarityMatrix(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries\MARScore_output\SimMat\simmat.csv", ms.similarity_matrices[0])